In [7]:
import cv2
import mediapipe as mp
import numpy as np

# Init webcam
cap = cv2.VideoCapture(0)

# Init MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Canvas for drawing
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Initial settings
draw_color = (255, 0, 255)
prev_x, prev_y = 0, 0
eraser_mode = False
colors = [(255, 0, 255), (0, 255, 0), (255, 255, 0), (0, 255, 255)]
color_index = 0


def get_finger_status(hand_landmarks):
    tips = [4, 8, 12, 16, 20]
    fingers = []

    # Thumb (use x)
    if hand_landmarks.landmark[tips[0]].x < hand_landmarks.landmark[tips[0]-1].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other fingers (use y)
    for tip in tips[1:]:
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers


while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)
            lm = handLms.landmark
            h, w, _ = img.shape
            x = int(lm[8].x * w)
            y = int(lm[8].y * h)

            fingers = get_finger_status(handLms)
            total_fingers = sum(fingers)

            if fingers[1] == 1 and sum(fingers) == 1:
                # ✏️ Drawing Mode (only index up)
                if prev_x == 0 and prev_y == 0:
                    prev_x, prev_y = x, y
                thickness = 20 if not eraser_mode else 50
                color = draw_color if not eraser_mode else (0, 0, 0)
                cv2.line(canvas, (prev_x, prev_y), (x, y), color, thickness)
                prev_x, prev_y = x, y

            elif sum(fingers) == 2:
                # ✌️ Stop drawing
                prev_x, prev_y = 0, 0
                print("✋ Drawing paused")

            elif sum(fingers) == 3:
                # 🎨 Switch color
                color_index = (color_index + 1) % len(colors)
                draw_color = colors[color_index]
                eraser_mode = False
                print("🎨 Color changed")

            elif sum(fingers) == 5:
                # 🧼 Clear canvas
                canvas = np.zeros((480, 640, 3), dtype=np.uint8)
                print("🧼 Canvas Cleared")

            else:
                prev_x, prev_y = 0, 0


    # Combine canvas with webcam feed
    img = cv2.addWeighted(img, 0.5, canvas, 0.5, 0)

    # Show current mode on screen
    cv2.rectangle(img, (0, 0), (640, 40), (0, 0, 0), -1)
    mode_text = "Eraser" if eraser_mode else "Draw"
    cv2.putText(img, f"Mode: {mode_text} | Color: {draw_color}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Virtual Painter", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1743946484.174508   87025 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743946484.175422   89667 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.0.1), renderer: Mesa Intel(R) Iris(R) Plus Graphics (ICL GT2)
W0000 00:00:1743946484.196839   89661 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743946484.215020   89662 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paused
✋ Drawing paus

KeyboardInterrupt: 